In [2]:

!wget  http://www.manythings.org/anki/fra-eng.zip
!unzip -q fra-eng.zip -d /content/french-eng

--2025-02-04 16:00:51--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip.1’

fra-eng.zip.1       100%[===================>]   7.57M  14.8MB/s    in 0.5s    

2025-02-04 16:00:52 (14.8 MB/s) - ‘fra-eng.zip.1’ saved [7943074/7943074]

replace /content/french-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/french-eng/fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [3]:
## importing libraries
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [4]:
batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path='/content/french-eng/fra.txt'

In [7]:
## vectorise the data
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text,target_text,_=line.split('\t')
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [8]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Wow!',
 'Wow!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Fire!',
 'Help!',
 'Hide.',
 'Hide.',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'Hello.',
 'Hello.',
 'Hello.',
 'Hello.',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Sorry?',
 'Attack!',
 'Attack!',
 'Attack!',
 'Attack!',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Exhale.',
 'Get up.',
 'Get up.',
 'Ge

In [9]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tEn route !\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tWaouh\xa0!\n',
 '\tWah\xa0!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\t

In [10]:
print("The no of inpur samples",len(input_texts))
print("The no of target samples",len(target_texts))

The no of inpur samples 10000
The no of target samples 10000


In [11]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

print("Number of unique input tokens:",num_encoder_tokens)
print("Number of unique output tokens:",num_decoder_tokens)
print("Max sequence length for inputs:",max_encoder_seq_length)
print("Max sequence length for outputs:",max_decoder_seq_length)

Number of unique input tokens: 70
Number of unique output tokens: 91
Max sequence length for inputs: 14
Max sequence length for outputs: 59


In [12]:
input_token_index=dict(
    [(char,i) for i,char in enumerate(input_characters)])

target_token_index=dict(
    [(char,i) for i,char in enumerate(target_characters)])

In [13]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [23]:
## One hot encoding

encoder_inputd=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
    dtype='float32')
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32')

decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32')


In [24]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_inputd[i,t,input_token_index[char]]=1
    encoder_inputd[i,t+1:,input_token_index[' ']]=1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1
    decoder_input_data[i,t+1:,target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1

In [18]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [19]:
encoder_input=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_input)
## current & prev input states of lstm
encoder_states=[state_h,state_c]

In [20]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [25]:
model=Model([encoder_inputd,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

ValueError: All `inputs` values must be KerasTensors. Received: inputs=[array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],

In [27]:
#trainin
model.fit([encoder_inputd,decoder_input_data],decoder_target_data,
          batch_size=batch_size,
          epochs=10,validation_split=0.2)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 50s 402ms/step - accuracy: 0.7383 - loss: 1.1524 - val_accuracy: 0.7140 - val_loss: 1.0490
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 83s 411ms/step - accuracy: 0.7498 - loss: 0.9232 - val_accuracy: 0.7301 - val_loss: 0.9563
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 84s 428ms/step - accuracy: 0.7678 - loss: 0.8394 - val_accuracy: 0.7590 - val_loss: 0.8415
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 422ms/step - accuracy: 0.7939 - loss: 0.7412 - val_accuracy: 0.7782 - val_loss: 0.7704
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 80s 403ms/step - accuracy: 0.8052 - loss: 0.6753 - val_accuracy: 0.7878 - val_loss: 0.7192
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 56s 452ms/step - accuracy: 0.8004 - loss: 0.7159 - val_accuracy: 0.7998 - val_loss: 0.6888
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 77s 414ms/step - accuracy: 0.8227 - loss: 0.6055 - val_accuracy: 0.8042 - val_loss: 0.6767
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 410ms/step - accuracy: 0.8299 - loss: 0

In [28]:
# During inference
# re-define encoder and decoder model
encoder_model = Model(encoder_input, encoder_states)
latent_dimension=256
decoder_state_inputs_h = Input(shape=(latent_dimension,))
decoder_state_inputs_c = Input(shape=(latent_dimension,))
decoder_state_inputs = [decoder_state_inputs_h, decoder_state_inputs_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_state_inputs)
decoder_state = [state_h, state_c]
decoder_model = Model(
    [decoder_inputs] + decoder_state_inputs,
    [decoder_outputs] + decoder_state)

In [29]:
## reverse look up

reverse_input_char_index=dict(
    (i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict(
    (i,char) for char,i in target_token_index.items())



In [30]:
reverse_input_char_index

{0: ' ',
 1: '!',
 2: '"',
 3: '$',
 4: '%',
 5: '&',
 6: "'",
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '5',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: '?',
 20: 'A',
 21: 'B',
 22: 'C',
 23: 'D',
 24: 'E',
 25: 'F',
 26: 'G',
 27: 'H',
 28: 'I',
 29: 'J',
 30: 'K',
 31: 'L',
 32: 'M',
 33: 'N',
 34: 'O',
 35: 'P',
 36: 'Q',
 37: 'R',
 38: 'S',
 39: 'T',
 40: 'U',
 41: 'V',
 42: 'W',
 43: 'Y',
 44: 'a',
 45: 'b',
 46: 'c',
 47: 'd',
 48: 'e',
 49: 'f',
 50: 'g',
 51: 'h',
 52: 'i',
 53: 'j',
 54: 'k',
 55: 'l',
 56: 'm',
 57: 'n',
 58: 'o',
 59: 'p',
 60: 'q',
 61: 'r',
 62: 's',
 63: 't',
 64: 'u',
 65: 'v',
 66: 'w',
 67: 'x',
 68: 'y',
 69: 'z'}

In [34]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [37]:
# for seq_index in range(100):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_inputd[seq_index: seq_index + 1]
#     decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decode_sequence(input_seq))